# Check Guild

In [1]:
import os, sys, random
import pandas as PD

In [2]:
BACKEND_PATH = os.path.dirname(os.path.dirname(os.getcwd()))
APPPROJ_PATH = os.path.join(BACKEND_PATH, 'app_proj')
sys.path.append(APPPROJ_PATH)
import notebooks as NT
print([x for x in dir(NT) if x[0]!='_' and len(x)>8])

['DataframeToDicts', 'GetRandom', 'NOTEBOOK_ENV']


In [3]:
import django
os.environ['DEPLOYMENT_ENV'] = NT.NOTEBOOK_ENV
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'True'
MODULE_PATH = os.path.dirname(os.getcwd())
os.chdir(os.path.dirname(MODULE_PATH))
django.setup()

env: DEV


In [4]:
import database as DB
print([x for x in dir(DB) if x[0]!='_' and len(x)>8])

['BaseManager', 'DeleteTable', 'GetNativeTableCounts', 'GetTableCounts', 'GetTableDictionary', 'InsertBulk', 'InsertSingle']


In [5]:
import django.db.models as JM
from django.contrib.auth import get_user_model

In [6]:
import emporium.models as EM
import engine.models as GM

In [7]:
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

### Guild Properties

In [8]:
userMd = get_user_model().objects.filter(user_name='Admin')[0]
userMd

<User: admin@thieves-guild.com>

In [9]:
guildMd = GM.Guild.objects.GetOrNone(UserFK=userMd, Name='123')
PD.DataFrame([guildMd.__dict__]).drop(['id', '_state'], axis=1, errors='ignore')

,UserFK_id,Name,LastPlayed,CreateDate,Selected,ThroneLevel,CampaignWorld,TotalPower,StorageGold,StorageGems,StorageWood,StorageStone,StorageIron,VaultGold,VaultGems,VaultWood,VaultStone,VaultIron
0,1,123,2024-03-13,2024-03-13,True,1,1,352,0,0,0,0,0,0,0,0,0,0


In [16]:
thieves = GM.ThiefInGuild.objects.filter(GuildFK=guildMd).values()
PD.DataFrame(thieves).drop(['id',
        'Name', 'BasePower', 'Experience', 
        'Position', 'Wounds', 'Cooldown',
        'BaseAgi', 'BaseCun', 'BaseMig', 'BaseEnd', 
        'TrainedAgi', 'TrainedCun', 'TrainedMig', 'TrainedEnd', 
        'Attack', 'Damage', 'Defense',
        'Traverse', 'Sabotage', 'Perceive', 
        ], axis=1, errors='ignore')

,GuildFK_id,Class,Stars,Level,Power,Agility,Cunning,Might,Endurance,Health,Status,CooldownExpire
0,12,Burglar,1,1,48,3,0,0,0,58,Ready,NaT
1,12,Scoundrel,1,1,64,0,3,0,0,58,Ready,NaT
2,12,Ruffian,1,1,48,0,0,3,0,58,Ready,NaT
3,12,Burglar,1,1,48,3,0,0,0,58,Exploring,2024-06-27 15:08:54+00:00
4,12,Scoundrel,1,1,96,0,5,0,0,58,Ready,NaT
5,12,Ruffian,1,1,48,0,0,3,0,58,Ready,NaT


In [11]:
thiefMds = GM.ThiefInGuild.objects.filter(GuildFK=guildMd)
thiefLs = []

for md in thiefMds:

    thiefDx = md.__dict__
    thiefLs.append(thiefDx)
    
    thiefItems = GM.ItemInGuild.objects.filter(ThiefFK=md).values()
    thiefDx['ItemCount'] = len(thiefItems)
    
    slots = ['weapon', 'armor', 'head', 'hands', 'feet', ]
    
    for sl in slots:
        itemCheck = [x for x in thiefItems if x['Slot']==sl]
        
        if itemCheck:
            item = itemCheck[0]
            res = item.pop('GuildFK_id')
            res = item.pop('ThiefFK_id')
            if sl in ['weapon', 'armor']: stat = item['Trait'][:3]
            else:     stat = 'skl' if item['Skill'] else 'cmb'
            item['icon'] = f"{sl}-{stat}.png"
        else:
            item = {}
            item['id'] = -1
            item['icon'] = f"{sl}-empty.png"

        thiefDx[sl] = item


PD.DataFrame(thiefItems)

""


In [12]:
thiefDf = PD.DataFrame(thiefLs)
thiefDf = thiefDf.drop(['_state', 'GuildFK_id', 'BasePower', 'BaseAgi', 'BaseCun', 'BaseMig', 'BaseEnd', 
                        'TrainedAgi', 'TrainedCun', 'TrainedMig', 'TrainedEnd',
                        'Experience', 'Position', 'Wounds', 'Cooldown'], 
                    axis=1, errors='ignore')
thiefDf = thiefDf.sort_values('Power', ascending=False)
thiefDf

,id,Name,Class,Stars,Level,Power,Agility,Cunning,Might,Endurance,...,Perceive,Traverse,Status,CooldownExpire,ItemCount,weapon,armor,head,hands,feet
4,55,Rigormorta,Scoundrel,1,1,96,0,5,0,0,...,0,0,Ready,NaT,2,"{'id': 148, 'Slot': 'weapon', 'Name': 'Blackth...","{'id': 149, 'Slot': 'armor', 'Name': 'Vestment...","{'id': -1, 'icon': 'head-empty.png'}","{'id': -1, 'icon': 'hands-empty.png'}","{'id': -1, 'icon': 'feet-empty.png'}"
1,52,Arawen,Scoundrel,1,1,64,0,3,0,0,...,5,0,Ready,NaT,1,"{'id': -1, 'icon': 'weapon-empty.png'}","{'id': -1, 'icon': 'armor-empty.png'}","{'id': 152, 'Slot': 'head', 'Name': 'Cowl', 'L...","{'id': -1, 'icon': 'hands-empty.png'}","{'id': -1, 'icon': 'feet-empty.png'}"
0,51,Nutani,Burglar,1,1,48,3,0,0,0,...,0,0,Ready,NaT,0,"{'id': -1, 'icon': 'weapon-empty.png'}","{'id': -1, 'icon': 'armor-empty.png'}","{'id': -1, 'icon': 'head-empty.png'}","{'id': -1, 'icon': 'hands-empty.png'}","{'id': -1, 'icon': 'feet-empty.png'}"
2,53,Popeska,Ruffian,1,1,48,0,0,3,0,...,0,0,Ready,NaT,0,"{'id': -1, 'icon': 'weapon-empty.png'}","{'id': -1, 'icon': 'armor-empty.png'}","{'id': -1, 'icon': 'head-empty.png'}","{'id': -1, 'icon': 'hands-empty.png'}","{'id': -1, 'icon': 'feet-empty.png'}"
3,54,Yortec,Burglar,1,1,48,3,0,0,0,...,0,0,Exploring,2024-06-27 15:08:54+00:00,0,"{'id': -1, 'icon': 'weapon-empty.png'}","{'id': -1, 'icon': 'armor-empty.png'}","{'id': -1, 'icon': 'head-empty.png'}","{'id': -1, 'icon': 'hands-empty.png'}","{'id': -1, 'icon': 'feet-empty.png'}"
5,56,Melchor,Ruffian,1,1,48,0,0,3,0,...,0,0,Ready,NaT,0,"{'id': -1, 'icon': 'weapon-empty.png'}","{'id': -1, 'icon': 'armor-empty.png'}","{'id': -1, 'icon': 'head-empty.png'}","{'id': -1, 'icon': 'hands-empty.png'}","{'id': -1, 'icon': 'feet-empty.png'}"


In [13]:
guildItems = GM.ItemInGuild.objects.filter(GuildFK=guildMd).values()
PD.DataFrame(guildItems).drop(['id'], axis=1, errors='ignore').sort_values('Slot')

,GuildFK_id,ThiefFK_id,Slot,Name,Level,TotalLv,Power,Requirement,Trait,Skill,Combat,Magic
1,12,NaN,armor,Brigandine,1,1,24,Burglar,agi 1,None,def 3,None
3,12,NaN,armor,Vestment,1,1,24,Scoundrel,cun 1,None,def 2,None
5,12,NaN,armor,Chain Maille,1,1,24,Ruffian,mig 1,None,def 1,None
7,12,NaN,armor,Brigandine,1,1,24,Burglar,agi 1,None,def 3,None
9,12,55.0,armor,Vestment,1,1,24,Scoundrel,cun 1,None,def 2,None
11,12,NaN,armor,Chain Maille,1,1,24,Ruffian,mig 1,None,def 1,None
17,12,NaN,feet,Greaves,1,1,16,None,None,None,att 1,None
16,12,NaN,feet,Boots,1,1,16,None,None,tra 5,None,None
15,12,NaN,hands,Gauntlet,1,1,16,None,None,None,dmg 1,None
14,12,NaN,hands,Gloves,1,1,16,None,None,sab 5,None,None


### Stages

In [14]:
towers = GM.GuildTower.objects.all().values()
PD.DataFrame(towers).drop(['id'], axis=1, errors='ignore')

AttributeError: module 'engine.models' has no attribute 'GuildTower'

### Stores